# Match analysis
## Database
- database 1: https://sports-statistics.com/sports-data/nba-basketball-datasets-csv-files/
- database 2: https://www.basketball-reference.com/

## Input 
- data/2019-20_pbp.csv

## Aims
1. split the whole records into individual matches
2. analyze the moves per team

## Output
1. matches/1-1143.csv
2. intensity.csv


In [1]:
import pandas as pd
import numpy as np
import copy

Check the data:

In [2]:
data_pbp = pd.read_csv('data/2019-20_pbp.csv')
display(data_pbp, data_pbp.columns)

,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,...,EnterGame,LeaveGame,TurnoverPlayer,TurnoverType,TurnoverCause,TurnoverCauser,JumpballAwayPlayer,JumpballHomePlayer,JumpballPoss,Unnamed: 40
0,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,720,NOP,Jump ball: D. Favors vs. M. Gasol (L. Ball gai...,...,NaN,NaN,NaN,NaN,NaN,NaN,D. Favors - favorde01,M. Gasol - gasolma01,L. Ball - balllo01,NaN
1,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,708,NOP,L. Ball misses 2-pt jump shot from 11 ft,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,Offensive rebound by D. Favors,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,D. Favors makes 2-pt layup at rim,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,689,NOP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539260,/boxscores/202010110MIA.html,playoff,The Arena Bay Lake Florida,October 11 2020,7:30 PM,LAL,4,29,LAL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
539261,/boxscores/202010110MIA.html,playoff,The Arena Bay Lake Florida,October 11 2020,7:30 PM,LAL,4,20,LAL,D. Howard makes 3-pt jump shot from 30 ft (ass...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
539262,/boxscores/202010110MIA.html,playoff,The Arena Bay Lake Florida,October 11 2020,7:30 PM,LAL,4,15,LAL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
539263,/boxscores/202010110MIA.html,playoff,The Arena Bay Lake Florida,October 11 2020,7:30 PM,LAL,4,0,LAL,End of 4th quarter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Index(['URL', 'GameType', 'Location', 'Date', 'Time', 'WinningTeam', 'Quarter',
       'SecLeft', 'AwayTeam', 'AwayPlay', 'AwayScore', 'HomeTeam', 'HomePlay',
       'HomeScore', 'Shooter', 'ShotType', 'ShotOutcome', 'ShotDist',
       'Assister', 'Blocker', 'FoulType', 'Fouler', 'Fouled', 'Rebounder',
       'ReboundType', 'ViolationPlayer', 'ViolationType', 'TimeoutTeam',
       'FreeThrowShooter', 'FreeThrowOutcome', 'FreeThrowNum', 'EnterGame',
       'LeaveGame', 'TurnoverPlayer', 'TurnoverType', 'TurnoverCause',
       'TurnoverCauser', 'JumpballAwayPlayer', 'JumpballHomePlayer',
       'JumpballPoss', 'Unnamed: 40'],
      dtype='object')

The number of matches:

In [3]:
print(np.sum(data_pbp['AwayPlay'].values=='End of Game'))
print(np.sum(data_pbp['HomePlay'].values=='End of Game'))

1143
0


The end signs are recorded in 'AwayPlay'

To split the whole records into individual matches:

In [4]:
ends = np.where(data_pbp['AwayPlay'].values=='End of Game')
display(ends)

(array([   564,   1025,   1515, ..., 538384, 538822, 539264], dtype=int64),)

In [5]:
start = 0
game = 1
for end in ends[0]:
    df_m = data_pbp.iloc[start:end+1]
    df_m.to_csv('data/matches/'+str(game)+'.csv')
    start = end+1
    game += 1

compute the sum of mones per team:

In [6]:
intensity_dict = {}
for i in range(1, 1144):
    df_match = pd.read_csv('data/matches/'+str(i)+'.csv')
    team_away = df_match.loc[0]['AwayTeam']
    team_home = df_match.loc[0]['HomeTeam']
    if team_away not in intensity_dict:
        intensity_dict[team_away]={'move': 0, 'sum': 0,\
                                  'move_q1': 0,\
                                  'move_q2': 0,\
                                  'move_q3': 0,\
                                  'move_q4': 0,\
                                  'move_q5': 0,\
                                  'move_q6': 0}
    if team_home not in intensity_dict:
        intensity_dict[team_home]={'move': 0, 'sum': 0,\
                                  'move_q1': 0,\
                                  'move_q2': 0,\
                                  'move_q3': 0,\
                                  'move_q4': 0,\
                                  'move_q5': 0,\
                                  'move_q6': 0}
    
    intensity_dict[team_away]['sum']+=1
    intensity_dict[team_home]['sum']+=1
    
    for index, row in df_match.iterrows():
        if pd.isnull(row['AwayPlay']) == False and \
        row['AwayPlay'].split()[0]+row['AwayPlay'].split()[1] == 'Jumpball:':
#             print(row['AwayPlay'])
            continue
        if pd.isnull(row['AwayPlay']) == False and \
        row['AwayPlay'].split()[0] == 'End':
#             print(row['AwayPlay'])
            continue
        if pd.isnull(row['AwayPlay']) == False:
            intensity_dict[team_away]['move']+=1
            mqstr = 'move_q'+str(row['Quarter'])
            intensity_dict[team_away][mqstr]+=1
            continue
        if pd.isnull(row['HomePlay']) == False:
            intensity_dict[team_home]['move']+=1
            mqstr = 'move_q'+str(row['Quarter'])
            intensity_dict[team_home][mqstr]+=1
            continue

In [18]:
display(intensity_dict)

{'NOP': {'move': 17055,
  'sum': 72,
  'move_q1': 4034,
  'move_q2': 4346,
  'move_q3': 4215,
  'move_q4': 4282,
  'move_q5': 178,
  'move_q6': 0},
 'TOR': {'move': 18890,
  'sum': 83,
  'move_q1': 4514,
  'move_q2': 4808,
  'move_q3': 4556,
  'move_q4': 4888,
  'move_q5': 96,
  'move_q6': 28},
 'LAL': {'move': 21564,
  'sum': 92,
  'move_q1': 5209,
  'move_q2': 5665,
  'move_q3': 5246,
  'move_q4': 5395,
  'move_q5': 49,
  'move_q6': 0},
 'LAC': {'move': 20422,
  'sum': 85,
  'move_q1': 4791,
  'move_q2': 5352,
  'move_q3': 4993,
  'move_q4': 5153,
  'move_q5': 104,
  'move_q6': 29},
 'CHI': {'move': 14712,
  'sum': 65,
  'move_q1': 3457,
  'move_q2': 3721,
  'move_q3': 3581,
  'move_q4': 3901,
  'move_q5': 52,
  'move_q6': 0},
 'CHO': {'move': 14753,
  'sum': 65,
  'move_q1': 3519,
  'move_q2': 3780,
  'move_q3': 3571,
  'move_q4': 3726,
  'move_q5': 138,
  'move_q6': 19},
 'DET': {'move': 14873,
  'sum': 66,
  'move_q1': 3535,
  'move_q2': 3807,
  'move_q3': 3639,
  'move_q4': 3786,

Compute the average move:

In [19]:
intensity_dict_m = copy.deepcopy(intensity_dict)

In [20]:
for team in intensity_dict_m:
    intensity_dict_m[team].pop('move_q5', None)
    intensity_dict_m[team].pop('move_q6', None)
    intensity_dict_m[team]['team'] = team
    intensity_dict_m[team]['move'] = intensity_dict_m[team]['move']/intensity_dict_m[team]['sum']
    intensity_dict_m[team]['move_q1'] = intensity_dict_m[team]['move_q1']/intensity_dict_m[team]['sum']
    intensity_dict_m[team]['move_q2'] = intensity_dict_m[team]['move_q2']/intensity_dict_m[team]['sum']
    intensity_dict_m[team]['move_q3'] = intensity_dict_m[team]['move_q3']/intensity_dict_m[team]['sum']
    intensity_dict_m[team]['move_q4'] = intensity_dict_m[team]['move_q4']/intensity_dict_m[team]['sum']

In [21]:
display(intensity_dict_m)

{'NOP': {'move': 236.875,
  'sum': 72,
  'move_q1': 56.02777777777778,
  'move_q2': 60.361111111111114,
  'move_q3': 58.541666666666664,
  'move_q4': 59.47222222222222,
  'team': 'NOP'},
 'TOR': {'move': 227.59036144578315,
  'sum': 83,
  'move_q1': 54.3855421686747,
  'move_q2': 57.9277108433735,
  'move_q3': 54.89156626506024,
  'move_q4': 58.89156626506024,
  'team': 'TOR'},
 'LAL': {'move': 234.3913043478261,
  'sum': 92,
  'move_q1': 56.619565217391305,
  'move_q2': 61.57608695652174,
  'move_q3': 57.02173913043478,
  'move_q4': 58.641304347826086,
  'team': 'LAL'},
 'LAC': {'move': 240.25882352941176,
  'sum': 85,
  'move_q1': 56.36470588235294,
  'move_q2': 62.96470588235294,
  'move_q3': 58.741176470588236,
  'move_q4': 60.62352941176471,
  'team': 'LAC'},
 'CHI': {'move': 226.33846153846153,
  'sum': 65,
  'move_q1': 53.184615384615384,
  'move_q2': 57.246153846153845,
  'move_q3': 55.09230769230769,
  'move_q4': 60.01538461538462,
  'team': 'CHI'},
 'CHO': {'move': 226.969230

create DataFrame:

In [22]:
for key in intensity_dict_m['CLE']:
    print(key)

move
sum
move_q1
move_q2
move_q3
move_q4
team


In [23]:
df_inten = pd.DataFrame(columns=['team', 'move', 'move_q1', 'move_q2', 'move_q3', 'move_q4', ])
for team in intensity_dict_m:
    df_inten = df_inten.append(intensity_dict_m[team], ignore_index=True)

remove the value of the minimum move (to show the results better)

In [24]:
# np.max(df_inten['make'].values)
df_inten = df_inten.drop(columns='sum')
for i in df_inten.columns:
    if i == 'team':
        continue
    else:
        df_inten[i] = df_inten[i].values-np.min(df_inten[i].values)

In [25]:
display(df_inten)

,team,move,move_q1,move_q2,move_q3,move_q4
0,NOP,17.952922,4.235570,4.776696,4.697511,3.134560
1,TOR,8.668284,2.593334,2.343295,1.047410,2.553904
2,LAL,15.469226,4.827357,5.991671,3.177583,2.303642
3,LAC,21.336746,4.572498,7.380290,4.897021,4.285867
4,CHI,7.416384,1.392408,1.661738,1.248152,3.677722
5,CHO,8.047153,2.346254,2.569431,1.094306,0.985415
6,DET,6.426407,1.768398,2.097403,1.292208,1.025974
7,IND,0.000000,0.000000,0.000000,0.000000,0.000000
8,CLE,7.385614,3.469331,1.215584,1.525075,0.446953
9,ORL,8.449717,2.412920,3.005328,2.130203,1.572594


In [27]:
df_inten.to_csv('data/intensity.csv')